In [ ]:
import pandas as pd

# import scipy
# from scipy.stats import t
# import seaborn as sns
# from datetime import datetime, timedelta
# import plotly.express as px
# from pathlib import Path
# import statsmodels.api as sm
# from sklearn.preprocessing import StandardScaler
# from statsmodels.stats.multitest import multipletests
# from scipy.stats import f_oneway as anova
# from statsmodels.formula.api import ols 
  
# import math
# from select import select

#kernal must be restarted if these module change 
from SCP_plotter import SCP_plotter
plotter = SCP_plotter(write_output = True)
from SCP_processor import SCP_processor
processor = SCP_processor()

In [ ]:
#Method put files in, one dictionary for each analysis, if there is only one file or you are looking in all files, then you don't need @'s in your SETTINGS_FILE
WRITE_OUTPUT = True
USE_MaxLFQ = False
filelist = [
             {"input1":"proteinGroups.txt",
             "input2":"peptides.txt",
             "input3":"report.pg_matrix.tsv",
             "input4":"report.pr_matrix.tsv",
             "input5":"summary.txt",
             "process_app": "FragPipe_TMT"},
          
            ]

            #Order of files
                #For FragPipe
                    #input1= Protein with MBR
                    #input2= Peptide with MBR
                #For DIANN
                    #input1= pg_matrix
                    #input2= pr_matrix 
                    #input3= pg_matrix MS2
                    #input4= pr_matrix MS2
                #For PD
                    #input1= Protein 
                    #input2= Peptide Groups
                    #input3= blank
                    #input4= blank
                    #input5= inpit_files


SETTINGS_FILE = "settings_test_TMT.txt" #tab delimited
    #Format
    #Required Columns
        #Group Name	
        #filter_in: a string pattern contained in desired raw filenames in analysis followed by an @ + the file number (filelist index) to look in
        #filter_out: a string pattern contained in undesired raw filenames
    #Extra columns
        #can be used to group by color, layer, or x position in CV and ID graphs

In [ ]:
# define group names and assign analysis to different groups(web app version was done through GUI so
# steps are taken to make sure they have same output)
x = processor.read_files(grouped_input_files=filelist)




In [ ]:
data_obj = processor.outer_join_data_objects(x)


In [ ]:
saved_settings = processor.sort_runs(data_obj,SETTINGS_FILE)


In [ ]:
#Missing val plot plot(protein and peptides)
print(data_obj["run_metadata"])
plot_options={
            "mean label": "True",
            "error bar": "ci95", #None, stdev, or ci95
            "X Title": "Conditions",
            "cutoffs": [33.3,50,99],
            "Y Title": "Protein Identifications",
            "color": ["#3E6990", "#7aa8e6","#AABD8C", "#E9E3B4", "#C6878F", "#fac8d3","#F39B6D",  "pink",
                  "orange", "brown", "pink", "gray", "olive", "cyan", "black","red",
                  "yellow","green","blue","indigo","violet"],
            "width": 700,
            "height": 450,
            "font": dict(size=32, family="Arial black"),
            "ID mode": "stacked",#stacked
            "Group By X": "Conditions", #ID_Mode does MS2 vs MBR
        }
plot_options["plot_type"] = "2" # 1 is protein, 2 is peptide
figure ,_ ,_ =plotter.Missing_Values_Plots(data_obj, plot_options, saved_settings)
figure.write_image("greedy_missing_val.png", scale=10, width = 1000, height=1000)
figure.show()

In [ ]:
#ID plot(protein and peptides)
plot_options={
            "mean label": "True",
            "error bar": "ci95", #None, stdev, or ci95
            "X Title": "Conditions",
            "Y Title": "Protein Identifications",
            "color": ["#3E6990", "#7aa8e6","#AABD8C", "#E9E3B4", "#C6878F", "#fac8d3","#F39B6D",  "pink",
                  "orange", "brown", "pink", "gray", "olive", "cyan", "black","red",
                  "yellow","green","blue","indigo","violet"],
            "width": 700,
            "height": 450,
            "font": dict(size=32, family="Arial black"),
            "ID mode": "grouped_stacked",#grouped, total, MS2, grouped_stacked, stacked
            "Group By X": "Sample Amount", #ID_Mode does MS2 vs MBR
            "Group By Color": "Isolation Window",#ID_Mode does MS2 vs MBR
            "Group By Stack": "ID_Mode",#ID_Mode does MS2 vs MBR
        }
plot_options["plot_type"] = "2" # 1 is protein, 2 is peptide
figure ,_ ,_ =plotter.ID_plots(data_obj, plot_options, saved_settings)
figure.write_image("asdf.png", scale=10, width = 1000, height=1000)
figure.show()

In [ ]:
# CV violin plot

# print(data_obj["protein_other_info"])
plot_options={    
        "median label": "True", #only works for total and MS2, can be found in CV_summary if you set WRITE_OUTPUT = True
        "box": False,
        "X Title": "Conditions (protein)",
        "Y Title": "CV of Abundance (%)",
        "color": ["#3E6990", "#7aa8e6","#C6878F", "#fac8d3","#AABD8C", "#E9E3B4",  "#F39B6D", "#C6878F", "pink",
                  "orange", "brown", "pink", "gray", "olive", "cyan", "black","red",
                  "yellow","green","blue","indigo","violet"],
        "width": 700,
        "height": 450,
        "font": dict(size=16, family="Arial black"),
        "ylimits": [-10, 400],
        "CV mode": "total",#grouped, total, MS2, grouped_stacked, stacked
        "Group By X": "Conditions", #ID_Mode does MS2 vs MBR
        "Group By Color": "Acquisition Mode",#ID_Mode does MS2 vs MBR
        "Group By Stack": "ID_Mode" #ID_Mode does MS2 vs MBR
    }
plot_options["plot_type"] = 2 # 1 is protein, 2 is peptide
figure, _, _ =plotter.CV_plots(data_obj, plot_options, saved_settings)
figure.show()


In [ ]:
# ID Venns plot
plot_options={
            "compare groups": ["Library","Quant"],
            "title": "Venn Diagram (Protein)",
            "opacity": 0.75,
            "color": ["#3E6990", "#C6878F", "#AABD8C","#fac8d3","#7aa8e6", "#E9E3B4","#00FF00", "#FFFF00", "#FF0000", "yellow", "red",
                      "green", "purple", "orange", "brown", "pink",
                      "gray",  "olive", "cyan", "blue",  "black", ]
        }
plot_options["plot_type"] = 1 # 1 is protein, 2 is peptide
figure, _, _ =plotter.venns_plots(data_obj, plot_options, saved_settings)
figure.show()



In [ ]:
# Protein Ranked Abundance plot
plot_options={
            "reference_group": "Library",
            "compare groups": ["Quant"],
            "title": "Ranked Abundance",
            "color": ["blue", "red", "green", "black", "yellow", "purple",
                      "orange", "brown", "pink", "gray", "olive", "cyan"],
            "symbol": ['circle', 'circle', 'circle'],
            "marker_size": 8,
            "width": 700,
            "height": 450,
            "font": dict(size=16, family="Arial black"),
            "ylimits": [],
            "xlimits": [],
            "X Title": "Ranked Abundance",
            "Y Title": "Average Reporter Ion Abundance",
        }
figure, _, _ =plotter.Rank_Abundance_Plots(data_obj, plot_options, saved_settings)
figure.show()

In [ ]:
# Protein Abundance Volcano plot
plot_options={
            "compare groups": ["Quant","Library"],
            "title": "Volcano Plot",
            "X Title": "Log2 Fold Change",
            "Y Title": "-Log10(adjusted p-value)",
            "up color": "red",
            "down color": "blue",
            "all color": "gray",
            "width": 700,
            "height": 450,
            "font": dict(size=16, family="Arial black"),
            "ylimits": [],
            "xlimits": []
        }
figure, _, _ =plotter.volcano_plots(data_obj, plot_options, saved_settings)
figure.show()


In [ ]:
# Protein Abundance PCA plot
plot_options={
            "compare groups": ["Quant","Library"],
            "title": "PCA Analysis",
            "color": ["blue", "red", "green", "black", "yellow", "purple",
                      "orange", "brown", "pink", "gray", "olive", "cyan"],
            "symbol": ['circle', 'circle', 'circle'],
            "marker_size": 8,
            "width": 700,
            "height": 450,
            "font": dict(size=16, family="Arial black"),
            "ylimits": [],
            "xlimits": []
        }
figure, _, _ =plotter.PCA_plots(data_obj, plot_options, saved_settings)
figure.show()


In [ ]:
# Protein Abundance Heat Map
plot_options={
            "compare groups": ["Quant","Library"],
            "log2_transform": False,
            "significant_only": True,
            "alpha": 0.05, 
            "min_fold_change": 2
        }
figure, _, _ =plotter.heatmap_plots(data_obj, plot_options, saved_settings)
figure.show()

In [ ]:
# ID Venns plot
plot_options={
            "compare groups": ["Quant","Library"],
            "inclusion list": "proteins_symbols.csv",
            "title": "Venn Diagram (Protein)",
            "opacity": 0.75,
            "color": ["#3E6990", "#C6878F", "#AABD8C","#fac8d3","#7aa8e6", "#E9E3B4","#00FF00", "#FFFF00", "#FF0000", "yellow", "red",
                      "green", "purple", "orange", "brown", "pink",
                      "gray",  "olive", "cyan", "blue",  "black", ]
        }
plot_options["plot_type"] = 1 # 1 is protein, 2 is peptide
figure, _, _ =plotter.inclusion_venn_plots(data_obj, plot_options, saved_settings)
figure.show()

